In [2]:
!pip install ultralytics > /dev/null 2>&1

In [3]:
import cv2
from ultralytics import YOLO 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = YOLO('/kaggle/input/yolo-11-training-weights/best_YOLO_11.pt')

In [5]:
video_1='/kaggle/input/different-speed-of-tennis-data-set/1_x_input_video.mp4'
result = model.track(video_1,conf=0.1, save=True,verbose=False)

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 11.1s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs/detect/track


In [6]:

def count_sports_ball_detections(video_paths, conf=0.1, frame_skip_factors=None):
    ball_detection_counts = {}

    if frame_skip_factors is None:
        # Default frame skip factors for different video speeds (can be customized)
        frame_skip_factors = {
            # '0.25x': 8,   # Process every 8th frame for slow-motion videos
            # '0.50x': 4,   # Process every 4th frame for slower videos
            '1x': 1,      # Process every frame for real-time speed
        #     '1.5x': 1,    # Process every frame for faster videos
        #     '2x': 1,      # Process every frame
        #     '4x': 0.5     # Process every frame, may need additional processing to avoid missed detections
        }

    # Iterate over each video path
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ball_count = 0  # Counter for sports ball detections
        frame_id = 0  # To keep track of frame count
        video_name = video_path.split('/')[-1]  # Extract video file name

        # Identify video speed based on its name
        speed_label = video_name.split('_')[0]  # Extract the speed (e.g., '0.25x')
        frame_skip_factor = frame_skip_factors.get(speed_label, 1)  # Get corresponding frame skip factor

        # Process the video frame by frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Only process the frame if it's not skipped
            if frame_id % frame_skip_factor == 0:
                # Detect objects in the frame using YOLO
                results = model(frame, conf=conf, verbose=False)
                
                # Iterate over detected objects and count tennis balls
                for result in results:
                    for obj in result.boxes:
                        # Check if the label is 'tennis ball'
                        if 'tennis ball' in result.names[int(obj.cls)]:
                            ball_count += 1
            
            frame_id += 1

        # Store the count for this video
        ball_detection_counts[video_name] = ball_count

        # Release the video capture for this file
        cap.release()
    
    return ball_detection_counts

# List of video paths and speed factors
video_paths = [
    # '/kaggle/input/different-speed-of-tennis-data-set/0.25_x_input_video.mp4',  # 0.25x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/0.50_x_input_video.mp4',  # 0.5x speed video
    '/kaggle/input/different-speed-of-tennis-data-set/1_x_input_video.mp4',     # 1x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/1.5_x_input_video.mp4',   # 1.5x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/2_x_input_video.mp4',     # 2x speed video
    # '/kaggle/input/different-speed-of-tennis-data-set/4_x_input_video.mp4'      # 4x speed video
]

# Count 'sports ball' detections in each video
ball_counts = count_sports_ball_detections(video_paths)

# Output the detection counts
print("Sports ball detection counts by video format:")
for video, count in ball_counts.items():
    print(f"{video}: {count} detections")

Sports ball detection counts by video format:
1_x_input_video.mp4: 75 detections


In [9]:
from ultralytics import YOLO 
import cv2
import pickle
import pandas as pd

class BallTracker:
    def __init__(self,model_path):
        self.model = YOLO(model_path)

    def interpolate_ball_positions(self, ball_positions):
        ball_positions = [x.get(1,[]) for x in ball_positions]
        # convert the list into pandas dataframe
        df_ball_positions = pd.DataFrame(ball_positions,columns=['x1','y1','x2','y2'])

        # interpolate the missing values
        df_ball_positions = df_ball_positions.interpolate()
        df_ball_positions = df_ball_positions.bfill()

        ball_positions = [{1:x} for x in df_ball_positions.to_numpy().tolist()]

        return ball_positions

    def get_ball_shot_frames(self, ball_positions):
        ball_positions = [x.get(1, []) for x in ball_positions]
        df_ball_positions = pd.DataFrame(ball_positions, columns=['x1', 'y1', 'x2', 'y2'])

        df_ball_positions['ball_hit'] = 0
        df_ball_positions['mid_y'] = (df_ball_positions['y1'] + df_ball_positions['y2']) / 2
        df_ball_positions['mid_y_rolling_mean'] = df_ball_positions['mid_y'].rolling(window=5, min_periods=1).mean()
        df_ball_positions['delta_y'] = df_ball_positions['mid_y_rolling_mean'].diff()
        minimum_change_frames_for_hit = 25

        for i in range(1, len(df_ball_positions) - int(minimum_change_frames_for_hit * 1.2)):
            negative_position_change = df_ball_positions['delta_y'].iloc[i] > 0 and df_ball_positions['delta_y'].iloc[i + 1] < 0
            positive_position_change = df_ball_positions['delta_y'].iloc[i] < 0 and df_ball_positions['delta_y'].iloc[i + 1] > 0

            if negative_position_change or positive_position_change:
                change_count = 0
                for change_frame in range(i + 1, i + int(minimum_change_frames_for_hit * 1.2) + 1):
                    negative_position_change_following_frame = df_ball_positions['delta_y'].iloc[i] > 0 and df_ball_positions['delta_y'].iloc[change_frame] < 0
                    positive_position_change_following_frame = df_ball_positions['delta_y'].iloc[i] < 0 and df_ball_positions['delta_y'].iloc[change_frame] > 0

                    if negative_position_change and negative_position_change_following_frame:
                        change_count += 1
                    elif positive_position_change and positive_position_change_following_frame:
                        change_count += 1

                if change_count > minimum_change_frames_for_hit - 1:
                    df_ball_positions.loc[i, 'ball_hit'] = 1  # Use .loc to avoid SettingWithCopyWarning

        frame_nums_with_ball_hits = df_ball_positions[df_ball_positions['ball_hit'] == 1].index.tolist()
        return frame_nums_with_ball_hits


    def detect_frames(self,frames, read_from_stub=False, stub_path=None):
        ball_detections = []

        if read_from_stub and stub_path is not None:
            with open(stub_path, 'rb') as f:
                ball_detections = pickle.load(f)
            return ball_detections

        for frame in frames:
            player_dict = self.detect_frame(frame)
            ball_detections.append(player_dict)
        
        if stub_path is not None:
            with open(stub_path, 'wb') as f:
                pickle.dump(ball_detections, f)
        
        return ball_detections

    def detect_frame(self,frame):
        results = self.model.predict(frame,conf=0.05)[0]

        ball_dict = {}
        for box in results.boxes:
            result = box.xyxy.tolist()[0]
            ball_dict[1] = result
        
        return ball_dict

    def draw_bboxes(self,video_frames, player_detections):
        output_video_frames = []
        for frame, ball_dict in zip(video_frames, player_detections):
            # Draw Bounding Boxes
            for track_id, bbox in ball_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Ball ID: {track_id}",(int(bbox[0]),int(bbox[1] -10 )),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)
            output_video_frames.append(frame)
        
        return output_video_frames


In [10]:
import cv2

# Initialize the BallTracker with your model path
tracker = BallTracker(model_path='/kaggle/input/updated-train-result-best2/best (2).pt')

# Load the video frames
video_path = '/kaggle/input/test-data/Tennis_Test_1.mp4'
cap = cv2.VideoCapture(video_path)
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()

# Detect tennis balls in frames
ball_detections = tracker.detect_frames(frames)

# Interpolate ball positions
interpolated_positions = tracker.interpolate_ball_positions(ball_detections)

# Get frames with detected ball hits
hit_frames = tracker.get_ball_shot_frames(interpolated_positions)

# Draw bounding boxes on video frames
output_frames = tracker.draw_bboxes(frames, ball_detections)

# Save output video (optional)
out = cv2.VideoWriter('/kaggle/working/output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (output_frames[0].shape[1], output_frames[0].shape[0]))
for frame in output_frames:
    out.write(frame)
out.release()



0: 640x384 (no detections), 51.1ms
Speed: 2.2ms preprocess, 51.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 tennis balls, 46.8ms
Speed: 1.8ms preprocess, 46.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 tennis ball, 46.9ms
Speed: 2.7ms preprocess, 46.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 tennis ball, 46.8ms
Speed: 1.9ms preprocess, 46.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 tennis ball, 46.9ms
Speed: 2.9ms preprocess, 46.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 tennis ball, 44.5ms
Speed: 2.0ms preprocess, 44.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 tennis ball, 36.3ms
Speed: 3.0ms preprocess, 36.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 37.1ms
Speed: 2.1ms preprocess, 37.1ms inference, 0